# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [166]:
with open("real_estate_sales_data.txt", encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [167]:
from langchain.text_splitter import CharacterTextSplitter

In [169]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 80,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [170]:
docs = text_splitter.create_documents([real_estate_sales])

In [171]:
docs[0]

Document(page_content='\\d+\\.\n[顾客问题]：这束玫瑰花能保存多久？\n[销售回答]：如果您照顾得当，可以保持新鲜大约一周时间，玫瑰花是表达爱意的最佳选择哦。')

In [172]:
len(docs)
# print(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [173]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17",
    base_url="https://api.xiaoai.plus/v1"))

In [174]:
# query = "小区吵不吵"
query = "向日葵象征着什么？"

In [175]:
answer_list = db.similarity_search(query)

In [176]:
for ans in answer_list:
    print(ans.page_content + "\n")

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：这花适合送给朋友吗？
[销售回答]：非常适合，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。



In [177]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [178]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [179]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002024AC56920>, search_kwargs={'k': 3})

In [180]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。



In [188]:
docs = topK_retriever.get_relevant_documents("想给暗恋的女生送一束花，你有什么好的建议吗？")

In [189]:
for doc in docs:
    print(doc.page_content + "\n")

[顾客问题]：你们有适合送给恋人的花吗？
[销售回答]：当然有，红玫瑰和郁金香是送给恋人的最佳选择，表达爱情的甜蜜和美好。

[顾客问题]：你们的花适合送给男朋友吗？
[销售回答]：当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。

[顾客问题]：这束花适合送给男朋友吗？
[销售回答]：当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [190]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.75}
)

In [191]:

docs = retriever.get_relevant_documents(query)
if len(docs) > 0:
    for doc in docs:
        print(doc.page_content + "\n")
else:
    print("赠人玫瑰，手有余香。不是每一种心意都能找到最合适的花语，但是每一束花都让对方感受到你的真情实意。\n")


[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：你们有适合送给朋友的花吗？
[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。

[顾客问题]：这花适合送给朋友吗？
[销售回答]：非常适合，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。



### 提取向量数据库中的`销售回答`

In [192]:
docs = retriever.get_relevant_documents(query)

In [193]:
docs[0].page_content

'[顾客问题]：你们有适合送给朋友的花吗？\n[销售回答]：有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。'

In [194]:
docs[0].page_content.split("[销售回答]：")

['[顾客问题]：你们有适合送给朋友的花吗？\n', '有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。']

In [195]:
ans = docs[0].page_content.split("[销售回答]：")[-1]

In [196]:
ans

'有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。'

#### 尝试各种问题

In [197]:
from typing import List

def love_master(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]：")[-1] for doc in docs]

    return ans_list

In [200]:
query = "分手送什么花？"

print(love_master(query))

[]


D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [201]:
print(love_master(query, 0.75))

['当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。', '当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。', '非常适合，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。', '有的，向日葵和雏菊这些花象征着友情和欢乐，是送给朋友的最佳选择。']


In [202]:
query = "女孩子更喜欢什么花？"

print(f"score:0.8 ans: {love_master(query)}\n")
print(f"score:0.75 ans: {love_master(query, 0.75)}\n")
print(f"score:0.5 ans: {love_master(query, 0.5)}\n")

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。', '有的，小雏菊和郁金香都是很好的选择，它们颜色鲜艳，寓意纯真和快乐。', '有的，小雏菊和郁金香这些花颜色鲜艳，象征着纯真和快乐，非常适合送给孩子。', '有的，小雏菊和郁金香这些花颜色鲜艳，象征着纯真和快乐，非常适合送给孩子。']

score:0.5 ans: ['当然适合！玫瑰和向日葵都是很好的选择，玫瑰代表爱情，向日葵象征着阳光和希望。', '有的，小雏菊和郁金香都是很好的选择，它们颜色鲜艳，寓意纯真和快乐。', '有的，小雏菊和郁金香这些花颜色鲜艳，象征着纯真和快乐，非常适合送给孩子。', '有的，小雏菊和郁金香这些花颜色鲜艳，象征着纯真和快乐，非常适合送给孩子。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [203]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17",
    base_url="https://api.xiaoai.plus/v1", model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [204]:
qa_chain({"query": "康乃馨的花语是什么？"})

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '康乃馨的花语是什么？',
 'result': '康乃馨的花语通常是“母爱”或“爱与温暖”。不同颜色的康乃馨还有不同的含义，比如红色代表爱和钦佩，粉色象征感恩和母爱，白色则常常代表纯洁和尊重。'}

In [206]:
qa_chain({"query": "前女友结婚，我应该送什么花？"})

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '前女友结婚，我应该送什么花？',
 'result': '送花的选择可以根据你与前女友的关系和她的喜好来决定。常见的选择有：\n\n1. **玫瑰**：虽然玫瑰通常代表爱情，但可以选择黄色或白色的玫瑰，象征祝福和友谊。\n2. **百合**：象征纯洁和祝福，适合在婚礼这样的场合送出。\n3. **康乃馨**：常用于表达祝贺，可以选择粉色或白色的康乃馨。\n\n最重要的是，选择一种你认为能表达你祝福的花，并保持礼貌和尊重的态度。'}

In [207]:
print(love_master("前女友结婚，我应该送什么花？"))

[]


D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [208]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [209]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17",
    base_url="https://api.xiaoai.plus/v1", model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [210]:
qa_chain({"query": "孩子考上了哈尔滨佛系大学，我应该送什么花来表示祝贺？"})

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '孩子考上了哈尔滨佛系大学，我应该送什么花来表示祝贺？',
 'result': '可以选择送康乃馨、百合花或玫瑰花来表示祝贺。康乃馨象征着祝福和爱，百合花代表纯洁和美好，而玫瑰花则传达了祝贺与祝福的心意。根据孩子的喜好选择合适的花束会更有意义。'}

In [211]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [213]:
qa_chain({"query": "男朋友喜欢剪了个中分头，还穿着背带裤打篮球，很帅气，我该送他什么花来表达我对他的喜爱？"})

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '男朋友喜欢剪了个中分头，还穿着背带裤打篮球，很帅气，我该送他什么花来表达我对他的喜爱？',
 'result': '你可以考虑送他向日葵或者白玫瑰。向日葵象征着阳光和积极向上的态度，适合表达你对他帅气形象的赞赏；而白玫瑰则代表纯洁和真挚的爱，适合表达你对他的喜欢。选择你觉得最能表达你心意的花吧！'}

In [163]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [214]:
result = qa_chain({"query": "女神结婚了，我很不爽，应该送什么花表达我的不快？"})

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [215]:
result

{'query': '女神结婚了，我很不爽，应该送什么花表达我的不快？',
 'result': '如果你想表达不快，可能不太适合送花，因为花通常是表达祝福和喜悦的象征。建议考虑其他方式来表达你的感受，比如和朋友聊聊或者写下你的想法。如果你还是想送花，可以选择一些寓意比较复杂的花，比如黑玫瑰，象征着悲伤或失落，但这也可能会引起误解。总之，送花不一定是表达不快的最佳选择。'}